In [ ]:
#from hybridpredictmaize22.hybridpredictmaize22.snpCompression import *
from hybridpredictmaize22.GEMlearn import *
from hybridpredictmaize22.GEMdataset import *
from hybridpredictmaize22.snpCompression import *

from pathlib import Path
import os

import allel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
from sklearn.decomposition import PCA



from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim

import fastcore.all as fc
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter,itemgetter
from functools import partial
from copy import copy
from contextlib import contextmanager
from warnings import warn


In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler


#| export
class newGemDataset():
    """
    Pytorch Dataset which can be used with dataloaders for simple batching during training loops
    """
    def __init__(self,W,Y,G, def_device='cpu'):
        self.W = W
        self.SNP = G
        self.Y = Y
        self.device = def_device
        
    def __len__(self): return self.Y[0].shape[0]

    def __getitem__(self,idx):
      y = self.Y[0][idx]
      e = self.Y[1][idx]
      h = self.Y[2][idx]
      d = self.Y[3][idx]

      #weather
      w = self.W[1][np.where(self.W[0] == e)[0][0]]

      #snp
      g = snp_data[1][:,np.where(snp_data[0] == h)[0][0]]
      return y,g,w


#| export
class ST():
    """
    A class which will hold the secondary trait data for the entire dataset for pre-training purposes
    
    init
        yield_data -> pandas table
        testYear -> e.g. 2019. this will set all data from a given year as the Test Set
    """
    def __init__(self, yield_data, testYear):

        self.Te = yield_data.iloc[([str(testYear) in x for x in yield_data['Env']]),:].reset_index()
        self.Tr = yield_data.iloc[([str(testYear) not in x for x in yield_data['Env']]),:].reset_index()

        self.secondary_traits = [
               # 'Stand_Count_plants',
               # 'Pollen_DAP_days',
               # 'Silk_DAP_days',
               # 'Plant_Height_cm',
               # 'Ear_Height_cm',
                #'Root_Lodging_plants',
                #'Stalk_Lodging_plants',
               # 'Twt_kg_m3',
                'Yield_Mg_ha',
                #'Date_Harvested'
                ]
        
        self.setup_scaler()
        self.scale_data(self.Tr)
        self.scale_data(self.Te)

        self.make_arrays(self.Tr)
        self.make_arrays(self.Te, False)
    def setup_scaler(self):
        ss = MinMaxScaler()
        ss.fit(np.array(self.Tr[self.secondary_traits]))
        self.scaler = ss

    def scale_data(self,df):
        scaled_secondary = self.scaler.transform(np.array(df[self.secondary_traits]))
        for c,i in enumerate(self.secondary_traits):
            #print(i)
            df[i] = scaled_secondary[:,c]
    
    def plot_yields(self):
        for i in self.secondary_traits:
            plt.hist(self.Tr[i],density=True, label='Train',alpha=.5,bins=50)
            plt.hist(self.Te[i],density=True, label='Test',alpha=.5,bins=50)
            plt.legend()
            plt.title(i)
            plt.show()

    def make_arrays(self,df,train=True):
      df = np.array(df[self.secondary_traits]), np.array(df['Env']) , np.array(df['Hybrid']), np.array(df['Date_Planted'])
      if train:
        self.Tr = df
      else:
        self.Te= df

#| export
class newWT():
    """
    A class which will hold the weather data for the entire dataset for training purposes
    
    init
        weather_data -> pandas table
        testYear -> e.g. 2019. this will set all data from a given year as the Test Set
    """
    def __init__(self, weather_data, testYear):
        
        self.Te = weather_data.iloc[([str(testYear) in x for x in weather_data['Year']]),:].reset_index()
        self.Tr = weather_data.iloc[([str(testYear) not in x for x in weather_data['Year']]),:].reset_index()
            
        self.setup_scaler()
        self.scale_data(self.Tr)
        self.scale_data(self.Te)

        self.make_array(self.Tr)
        self.make_array(self.Te,False)
            
    def setup_scaler(self):
        ss = MinMaxScaler()
        ss.fit(self.Tr.select_dtypes('float'))
        self.scaler = ss
            
    def scale_data(self, df):
        fd = df.select_dtypes('float')
        fs = self.scaler.transform(fd)
        df[fd.columns] = fs

    def make_array(self, df,train = True):
      for c,i in enumerate(set(df['Env'])):
        env_weather = np.array(df[df['Env'] == i].iloc[:,4:-1])
        #print(env_weather.shape)
        if c == 0:
          env_order = list([i])
          weather_array =   np.array(df[df['Env'] == i].iloc[:,4:-1])
          weather_array = np.expand_dims(weather_array,axis=0)
        else:
          weather_array = np.vstack((weather_array, env_weather[None,:,:]))
          env_order.append(i)

        if train:
          self.Tr = (np.array(env_order), np.array(weather_array))
        else:
          self.Te = (np.array(env_order), np.array(weather_array))

In [ ]:
test_split = 2019
test_year=2019

path_snps = Path('./data/snpCompress/')
data_path = Path('./data/Training_Data/')
path_train_weatherTable =data_path/'4_Training_Weather_Data_2014_2021.csv'
path_train_yieldTable = data_path/'1_Training_Trait_Data_2014_2021.csv'
snp_compression = 'PCS_100'
batch_size = 64

snp_data = collect_snps(Path('./data/snpCompress/PCS_50/')) # Read in the SNP profiles
yield_data = pd.read_csv(path_train_yieldTable) # Read in trait data 
yield_data = yield_data[yield_data['Yield_Mg_ha'].notnull()] #Remove plots w/ missing yields
weather_data = pd.read_csv(path_train_weatherTable) # Read in Weather Data
weather_data['Year'] = [x.split('_')[1] for x in weather_data['Env']] #Store Year in a new column
#removes yield data where no weather data
setYield = set(yield_data['Env'])
setWeather = set(weather_data['Env'])
only_yield = setYield - setWeather
only_weather = setWeather - setYield
yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Env']],:]
#removes yield data where no genotype data
setSNP = set(snp_data[0])
setYield = set(yield_data['Hybrid'])
only_yield = setYield - setSNP
yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Hybrid']],:]

to_remove = []
for i in set(weather_data['Env']):
    for x in (weather_data.loc[weather_data['Env'] == i].index[300:]):
        to_remove.append(x)
        
weather_data_clean = weather_data.drop(index=to_remove)



#weather_data = remove_leapdays(weather_data)
weather_data_clean = weather_data_clean.reset_index()
yield_data=yield_data.sample(frac=1)
yield_data = yield_data.reset_index()

batch_size = 512

W = newWT(weather_data_clean,test_year)
Y=ST(yield_data,test_year)

tr_gem = newGemDataset(W=W.Tr,
                    Y = Y.Tr,
                    G = snp_data)
te_gem = newGemDataset(W=W.Te,
                    Y= Y.Te,
                    G = snp_data)

# tr_ds = GemDataset(gem.W.Tr, gem.Y.Tr, gem.SNP, def_device='cpu')
# te_ds = GemDataset(gem.W.Te, gem.Y.Te, gem.SNP, def_device ='cpu')

tr_dataloader = DataLoader(tr_gem, batch_size=batch_size, shuffle=True, num_workers=4)
te_dataloader = DataLoader(te_gem, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
W.Tr[1][0].shape

(300, 16)

In [ ]:
class EfficientNet2(nn.Module):
    def __init__(self, in_chan = 1 , num_classes=100):
        super(EfficientNet2, self).__init__()

        # Define the convolutional layers
        self.conv1 = nn.Conv1d(in_chan, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm1d(128)

        # Define the depthwise separable convolutional layers
        self.dwconv1 = nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1, groups=128, bias=False)
        self.bn4 = nn.BatchNorm1d(128)

        self.dwconv2 = nn.Conv1d(128, 128, kernel_size=3, stride=2, padding=1, groups=128, bias=False)
        self.bn5 = nn.BatchNorm1d(128)

        # Define the fully connected layer
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # Pass the input through the convolutional layers
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)

        # Pass the input through the depthwise separable convolutional layers
        x = self.dwconv1(x)
        x = self.bn4(x)
        x = F.relu(x)

        x = self.dwconv2(x)
        x = self.bn5(x)
        x = F.relu(x)

        # Flatten the feature maps before passing them through the fully connected layer
        # x = x.view(-1, 128)
        # x = self.fc(x)

        return x
    
class EfficientNet3(nn.Module):
    def __init__(self, in_chan = 1 , num_classes=100):
        super(EfficientNet3, self).__init__()

        # Define the convolutional layers
        self.conv1 = nn.Conv1d(in_chan, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm1d(128)

        # Define the depthwise separable convolutional layers
        self.dwconv1 = nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm1d(128)

        self.dwconv2 = nn.Conv1d(128, 128, kernel_size=3, stride=2, padding=1,  bias=False)
        self.bn5 = nn.BatchNorm1d(128)

        # Define the fully connected layer
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # Pass the input through the convolutional layers
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)

        # Pass the input through the depthwise separable convolutional layers
        x = self.dwconv1(x)
        x = self.bn4(x)
        x = F.relu(x)

        x = self.dwconv2(x)
        x = self.bn5(x)
        x = F.relu(x)

        # Flatten the feature maps before passing them through the fully connected layer
        # x = x.view(-1, 128)
        # x = self.fc(x)

        return x

class NNEnsemble3(torch.nn.Module):
  def __init__(self, hidden_list, models_list, alpha=None):
    super().__init__()

    self.models = models_list

    self.layers = nn.ModuleList()
    self.lin1 = torch.nn.Linear(hidden_list[0], hidden_list[1])
    self.lin2 = torch.nn.Linear(hidden_list[1], hidden_list[2])
    self.out = torch.nn.Linear(hidden_list[2], 1)

    
    if alpha != None:
        print('X')
        for c,l in enumerate(self.layers):
            print(l)
            torch.nn.init.xavier_normal_(l.weight,gain=alpha)
    self.out = nn.LazyLinear(1)

  def forward(self,x):
    g,w = x
    g = self.models[0](g)
    w = self.models[1](w)
 #   print(g.shape, w.shape)

    if w.dim() == 3:
      w = w.view(w.shape[0], w.shape[1] * w.shape[2])
    if g.dim() == 3:
      g = g.view(g.shape[0], g.shape[1] * g.shape[2])

    x = torch.concat((g,w),axis=1)
    for c,layer in enumerate(self.layers):
      x = layer(x)
      x = torch.nn.functional.relu(x) 
    return self.out(x)

In [ ]:
class EfficientNet2(nn.Module):
    def __init__(self, in_chan = 1 , num_classes=100):
        super(EfficientNet2, self).__init__()

        # Define the convolutional layers
        self.conv1 = nn.Conv1d(in_chan, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm1d(128)

        # Define the depthwise separable convolutional layers
        self.dwconv1 = nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1, groups=128, bias=False)
        self.bn4 = nn.BatchNorm1d(128)

        self.dwconv2 = nn.Conv1d(128, 128, kernel_size=3, stride=2, padding=1, groups=128, bias=False)
        self.bn5 = nn.BatchNorm1d(128)

        # Define the fully connected layer
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # Pass the input through the convolutional layers
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)

        # Pass the input through the depthwise separable convolutional layers
        x = self.dwconv1(x)
        x = self.bn4(x)
        x = F.relu(x)

        x = self.dwconv2(x)
        x = self.bn5(x)
        x = F.relu(x)

        # Flatten the feature maps before passing them through the fully connected layer
        # x = x.view(-1, 128)
        # x = self.fc(x)

        return x
    
class EfficientNet3(nn.Module):
    def __init__(self, in_chan = 1 , num_classes=100):
        super(EfficientNet3, self).__init__()

        # Define the convolutional layers
        self.conv1 = nn.Conv1d(in_chan, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm1d(128)

        # Define the depthwise separable convolutional layers
        self.dwconv1 = nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm1d(128)

        self.dwconv2 = nn.Conv1d(128, 128, kernel_size=3, stride=2, padding=1,  bias=False)
        self.bn5 = nn.BatchNorm1d(128)

        # Define the fully connected layer
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # Pass the input through the convolutional layers
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)

        # Pass the input through the depthwise separable convolutional layers
        x = self.dwconv1(x)
        x = self.bn4(x)
        x = F.relu(x)

        x = self.dwconv2(x)
        x = self.bn5(x)
        x = F.relu(x)

        # Flatten the feature maps before passing them through the fully connected layer
        # x = x.view(-1, 128)
        # x = self.fc(x)

        return x

class NNEnsemble3(torch.nn.Module):
  def __init__(self, hidden_list, models_list, alpha=None):
    super().__init__()

    self.models = models_list

    self.layers = nn.ModuleList()
    self.lin1 = torch.nn.Linear(hidden_list[0], hidden_list[1])
    self.lin2 = torch.nn.Linear(hidden_list[1], hidden_list[2])
    self.out = torch.nn.Linear(hidden_list[2], 1)

    
    if alpha != None:
        print('X')
        for c,l in enumerate(self.layers):
            print(l)
            torch.nn.init.xavier_normal_(l.weight,gain=alpha)
    self.out = nn.LazyLinear(1)

  def forward(self,x):
    g,w = x
    g = self.models[0](g)
    w = self.models[1](w)
 #   print(g.shape, w.shape)

    if w.dim() == 3:
      w = w.view(w.shape[0], w.shape[1] * w.shape[2])
    if g.dim() == 3:
      g = g.view(g.shape[0], g.shape[1] * g.shape[2])

    x = torch.concat((g,w),axis=1)
    for c,layer in enumerate(self.layers):
      x = layer(x)
      x = torch.nn.functional.relu(x) 
    return self.out(x)

def moving_average(arr, window_size):
    """Calculate the moving average of an array.
    
    Parameters:
    arr (np.ndarray): Input array with shape (n_samples,).
    window_size (int): Size of the moving window.
    
    Returns:
    np.ndarray: Moving average of the array, with shape (n_samples - window_size + 1,).
    """
    # Initialize a NumPy array to store the moving averages
    ma = np.zeros(len(arr) - window_size + 1)
    
    # Calculate the moving average
    for i in range(len(ma)):
        ma[i] = np.mean(arr[i:i+window_size])
        
    return ma


In [ ]:
gm = EfficientNet3(num_classes=1500)
wm = EfficientNet3(300, num_classes=1500)
                    

model = NNEnsemble3([2000,1000,1000], [gm,wm],alpha=2)
# plt.hist(detach_list(model((g,w))))

opt = optim.Adamax(model.parameters(), lr=.0005, weight_decay=.0001)
#scheduler =  optim.lr_scheduler.ReduceLROnPlateau(opt)

loss_func = torch.nn.functional.huber_loss

tr_loss = [] 
te_loss = []
te_MA = []
predicts = []
targets = []

#for i in tqdm(range(len(tr_dataloader))):
for i in tqdm(range(50)):
  model.train()
  #train loop
  y,g,w = next(iter(tr_dataloader))
  y =  y[:,-1]
  y = y.type(torch.float32)
  g = g.type(torch.float32)
  w = w.type(torch.float32)

  preds = model((g,w))
  preds = preds.squeeze(1)

  loss = loss_func(y, preds)

  loss.backward()
  opt.step()
  opt.zero_grad()
  #scheduler.step(loss)
  tr_loss.append(loss.cpu().detach().numpy())


  #test loop
  model.eval()
  y,g,w = next(iter(te_dataloader))
  y =  y[:,-1]
  y = y.to(torch.float32)
  g = g.to(torch.float32)
  w = w.to(torch.float32)

  preds = model((g,w))
  preds = preds.squeeze(1)

  loss = loss_func(y,preds)

  te_loss.append(loss.cpu().detach().numpy())

  if i % 100 == 0:
    print(i, loss)

  if i > 15:
    te_MA.append(moving_average(te_loss[i-15:],14)[-1])

    if i > 80:
        if ((te_MA[-int(round((0.45 * len(te_loss)),1))] < te_loss[i])):
            plt.plot(tr_loss)
            plt.plot(te_loss)
            plt.show()
            print('')
            print('early')
            print(te_loss[-1])

            plt.plot(tr_loss)
            plt.plot(te_loss)
            plt.ylim(0,1)
            plt.show()

            plt.ylim(0,.056)
            plt.plot(tr_loss)
            plt.plot(te_loss)
            plt.show()

            break

  
  
# plt.plot(tr_loss)
# plt.plot(te_loss)
# plt.show()

X


  2%|█▋                                                                                 | 1/50 [00:01<01:01,  1.25s/it]

0 tensor(0.7951, grad_fn=<HuberLossBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:59<00:00,  1.20s/it]


In [ ]:
ys = []
predicted = []
for y,g,w in te_dataloader:
    model.eval()
    preds = model((g.type(torch.float32),w.type(torch.float32)))
    
    for i in preds:
        predicted.append(i.detach().numpy())
    for x in y:
        ys.append(x.detach().numpy())
        
mypred = np.array(predicted).ravel()
mytarget = np.array(ys)


rmse = []
for p,t in zip(mypred,mytarget):
    if not np.isnan(t[-1]) or np.isnan(p):
        target_scaled = (Y.scaler.inverse_transform(np.array(t).reshape(1,-1))[0][-1])
        pred_scaled = (Y.scaler.inverse_transform(np.array(p).reshape(-1,1)))
        rmse.append(np.sqrt((target_scaled-pred_scaled)**2))
    else:
        pass
print(np.mean(rmse))

In [ ]:
mypred = np.array(predicted).ravel()
mytarget = np.array(ys)

In [ ]:
import math 
rmse = []
for p,t in zip(mypred,mytarget):
    if not np.isnan(t[-1]) or np.isnan(p):
        target_scaled = (Y.scaler.inverse_transform(np.array(t).reshape(1,-1))[0][-1])
        pred_scaled = (Y.scaler.inverse_transform(np.array(p).reshape(-1,1)))
        rmse.append(np.sqrt((target_scaled-pred_scaled)**2))
    else:
        pass
print(np.mean(rmse))

3.5782666


In [ ]:
#df.drop(columns=[column_name], inplace=True)

weather_data_clean.drop(columns=['Year'],inplace=True)

In [ ]:
test_weather_data_raw

,Env,Date,QV2M,PS,WS2M,ALLSKY_SFC_SW_DWN,RH2M,GWETTOP,T2MWET,T2M_MAX,T2MDEW,GWETROOT,GWETPROF,T2M,T2M_MIN,ALLSKY_SFC_PAR_TOT,PRECTOTCORR,ALLSKY_SFC_SW_DNI
0,DEH1_2022,20220101,9.70,100.62,3.25,1.89,97.00,0.61,13.69,17.08,13.51,0.62,0.61,13.89,10.15,10.18,4.59,1.30
1,DEH1_2022,20220102,9.70,100.44,3.66,2.45,93.00,0.66,13.93,16.78,13.37,0.65,0.63,14.49,8.79,13.28,14.32,1.43
2,DEH1_2022,20220103,3.30,101.48,6.94,2.30,84.56,0.67,-0.85,7.39,-2.04,0.66,0.65,0.33,-5.74,12.02,25.85,2.05
3,DEH1_2022,20220104,2.14,102.81,1.91,10.73,85.75,0.66,-6.02,1.01,-7.28,0.67,0.65,-4.77,-9.42,51.66,0.01,25.53
4,DEH1_2022,20220105,4.76,101.36,2.99,5.67,98.44,0.70,3.15,5.08,3.10,0.68,0.66,3.21,-4.70,28.96,2.93,2.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8159,WIH3_2022,20221106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8160,WIH3_2022,20221107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8161,WIH3_2022,20221108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8162,WIH3_2022,20221109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
submission = pd.read_csv('./data/Testing_Data/1_Submission_Template_2022.csv')
#impute missing data
from sklearn.impute import SimpleImputer
col_order = weather_data_clean.columns
test_weather_data_raw = pd.read_csv('./data/Testing_Data/4_Testing_Weather_Data_2022.csv')
test_weather_data_raw = test_weather_data_raw.reindex(columns=col_order)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(test_weather_data_raw.select_dtypes('float'))
test_weather_data = imputer.transform(test_weather_data_raw.select_dtypes('float'))
test_weather_array = W.scaler.transform(test_weather_data)

wet_dict = {}
for i in set(submission['Env']):
  env_index = np.array(list(test_weather_data_raw.loc[test_weather_data_raw['Env'] == i].index[:300]))
  wet_dict[i] = test_weather_array[env_index,:]
    
#for i in range(submission.shape[0]):
preds = []
for i in range(submission.shape[0]):
  ENV, HYBRID = submission.iloc[i]['Env'], submission.iloc[i]['Hybrid']
  genotype_index = np.where(snp_data[0]=="BGEM-0124-N/LH244")[0][0]
  g = snp_data[1][:,167]
  w = wet_dict[ENV]

  g = torch.tensor(g)
  w = torch.tensor(w)

  gin = g.view(1,1,g.shape[0]).type(torch.float32)
  win = w.view(1,w.shape[0], w.shape[1]).type(torch.float32)

  preds.append(model((gin,win)))

/home/burbank/miniconda3/envs/fastai/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(

KeyboardInterrupt



In [ ]:
submission

In [ ]:
sub_template = './data/Testing_Data/1_Submission_Template_2022.csv'
test_weather = './data/Testing_Data/4_Testing_Weather_Data_2022.csv'

In [ ]:
#impute missing data
from sklearn.impute import SimpleImputer
test_weather_data_raw = pd.read_csv(test_weather)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(test_weather_data_raw.select_dtypes('float'))
test_weather_data = imputer.transform(test_weather_data)

test_weather_array = W.scaler.transform(test_weather_data)

In [ ]:
snp_data

In [ ]:
test_env = pd.read_csv(sub_template).iloc[3000,:]["Env"]
test_weather_array[test_weather_data_raw.loc[test_weather_data_raw['Env'] == test_env][:300].index].shape

In [ ]:
for i in pd.read_csv(sub_template)['Env']:
    test_weather_array[test_weather_data_raw.loc[test_weather_data_raw['Env'] == i][:300].index].shape

In [ ]:
test_weather_data_raw

In [ ]:
pd.read_csv(test_weather).head(50).select_dtypes('float')

In [ ]:
class EfficientNet2(nn.Module):
    def __init__(self, in_chan = 1 , num_classes=100):
        super(EfficientNet2, self).__init__()

        # Define the convolutional layers
        self.conv1 = nn.Conv1d(in_chan, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm1d(128)

        # Define the depthwise separable convolutional layers
        self.dwconv1 = nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1, groups=128, bias=False)
        self.bn4 = nn.BatchNorm1d(128)

        self.dwconv2 = nn.Conv1d(128, 128, kernel_size=3, stride=2, padding=1, groups=128, bias=False)
        self.bn5 = nn.BatchNorm1d(128)

        # Define the fully connected layer
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # Pass the input through the convolutional layers
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)

        # Pass the input through the depthwise separable convolutional layers
        x = self.dwconv1(x)
        x = self.bn4(x)
        x = F.relu(x)

        x = self.dwconv2(x)
        x = self.bn5(x)
        x = F.relu(x)

        # Flatten the feature maps before passing them through the fully connected layer
        # x = x.view(-1, 128)
        # x = self.fc(x)

        return x
    
class EfficientNet3(nn.Module):
    def __init__(self, in_chan = 1 , num_classes=100):
        super(EfficientNet3, self).__init__()

        # Define the convolutional layers
        self.conv1 = nn.Conv1d(in_chan, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm1d(128)

        # Define the depthwise separable convolutional layers
        self.dwconv1 = nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm1d(128)

        self.dwconv2 = nn.Conv1d(128, 128, kernel_size=3, stride=2, padding=1,  bias=False)
        self.bn5 = nn.BatchNorm1d(128)

        # Define the fully connected layer
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # Pass the input through the convolutional layers
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)

        # Pass the input through the depthwise separable convolutional layers
        x = self.dwconv1(x)
        x = self.bn4(x)
        x = F.relu(x)

        x = self.dwconv2(x)
        x = self.bn5(x)
        x = F.relu(x)

        # Flatten the feature maps before passing them through the fully connected layer
        # x = x.view(-1, 128)
        # x = self.fc(x)

        return x

class NNEnsemble3(torch.nn.Module):
  def __init__(self, hidden_list, models_list, alpha=None):
    super().__init__()

    self.models = models_list

    self.layers = nn.ModuleList()
    self.lin1 = torch.nn.Linear(hidden_list[0], hidden_list[1])
    self.lin2 = torch.nn.Linear(hidden_list[1], hidden_list[2])
    self.out = torch.nn.Linear(hidden_list[2], 1)

    
    if alpha != None:
        print('X')
        for c,l in enumerate(self.layers):
            print(l)
            torch.nn.init.xavier_normal_(l.weight,gain=alpha)
    self.out = nn.LazyLinear(1)

  def forward(self,x):
    g,w = x
    g = self.models[0](g)
    w = self.models[1](w)
 #   print(g.shape, w.shape)

    if w.dim() == 3:
      w = w.view(w.shape[0], w.shape[1] * w.shape[2])
    if g.dim() == 3:
      g = g.view(g.shape[0], g.shape[1] * g.shape[2])

    x = torch.concat((g,w),axis=1)
    for c,layer in enumerate(self.layers):
      x = layer(x)
      x = torch.nn.functional.relu(x) 
    return self.out(x)

In [ ]:
class LazyMLP(torch.nn.Module):
    def __init__(self, hidden_list, dummy, alpha=None, dropout=0.1):
        super().__init__()
        self.dropout = dropout

        # Create a list of linear layers, with the correct input and output dimensions
        self.layers = torch.nn.ModuleList([nn.LazyLinear(x) for x in hidden_list])
        
        
        if alpha != None:
            x = dummy
            for l in self.layers:
                if x.dim() == 3:
                    x = x.view(x.shape[0], x.shape[1]*x.shape[2])

                x = l(x)
                torch.nn.init.xavier_normal_(l.weight,gain=hidden_list[0] / 100)

            

    def forward(self, x):
        # Apply the dropout layer to the input
        #x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        if x.dim() == 3:
            x = x.view(x.shape[0], x.shape[1]*x.shape[2])
        # Iterate through the linear layers, applying each one to the input
        for c, layer in enumerate(self.layers):
            x = layer(x)
            
            if c < len(self.layers)-1:
              x = torch.nn.functional.relu(x)  
            
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)

        return x
    
    

class NNEnsemble(torch.nn.Module):
  def __init__(self, hidden_list, models_list, alpha=None):
    super().__init__()

    self.models = models_list

    self.layers = nn.ModuleList()
    self.lin1 = torch.nn.Linear(hidden_list[0], hidden_list[1])
    self.lin2 = torch.nn.Linear(hidden_list[1], hidden_list[2])
    self.out = torch.nn.Linear(hidden_list[2], 1)

    
    if alpha != None:
        print('X')
        for c,l in enumerate(self.layers):
            print(l)
            torch.nn.init.xavier_normal_(l.weight,gain=alpha)
    self.out = nn.LazyLinear(1)

  def forward(self,x):
    g,w = x
    g = self.models[0](g)
    w = self.models[1](w)

    if w.dim() == 3:
      w = w.view(w.shape[0], w.shape[1] * w.shape[2])
    x = torch.concat((g,w),axis=1)
    for c,layer in enumerate(self.layers):
      x = layer(x)
      x = torch.nn.functional.relu(x) 
    return self.out(x)

In [ ]:
gmodel = LazyMLP([500,50,25],2)
wmodel = LazyMLP([5840,50,15],1)
model = NNEnsemble([200,50,25], [gmodel,wmodel])
#plt.hist(detach_list(model((g,w))))

/home/burbank/miniconda3/envs/fastai/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
wmodel(w.type(torch.float32)).shape

torch.Size([249, 15])

In [ ]:
gmodel(g.type(torch.float32)).shape

torch.Size([249, 25])

In [ ]:
model((g.type(torch.float32), w.type(torch.float32)))

In [ ]:
# gm = EfficientNet3(num_classes=1500)
# wm = EfficientNet3(300, num_classes=1500)
                    
gmodel = LazyMLP([500,50,25],2)
wmodel = LazyMLP([5840,50,15],1)
model = NNEnsemble([200,100,50], [gmodel,wmodel])
#plt.hist(detach_list(model((g,w))))

# model = NNEnsemble3([2000,1000,1000], [gm,wm],alpha=2)
# # plt.hist(detach_list(model((g,w))))

opt = optim.Adamax(model.parameters(), lr=.0001, weight_decay=.0001)
#scheduler =  optim.lr_scheduler.ReduceLROnPlateau(opt)

loss_func = torch.nn.functional.huber_loss

tr_loss = [] 
te_loss = []
te_MA = []
predicts = []
targets = []

#for i in tqdm(range(len(tr_dataloader))):
for i in tqdm(range(5000)):
  model.train()
  #train loop
  y,g,w = next(iter(tr_dataloader))
  y =  y[:,-1]
  y = y.type(torch.float32)
  g = g.type(torch.float32)
  w = w.type(torch.float32)

  preds = model((g,w))
  preds = preds.squeeze(1)

  loss = loss_func(y, preds)

  loss.backward()
  opt.step()
  opt.zero_grad()
  #scheduler.step(loss)
  tr_loss.append(loss.cpu().detach().numpy())


  #test loop
  model.eval()
  y,g,w = next(iter(te_dataloader))
  y =  y[:,-1]
  y = y.to(torch.float32)
  g = g.to(torch.float32)
  w = w.to(torch.float32)

  preds = model((g,w))
  preds = preds.squeeze(1)

  loss = loss_func(y,preds)

  te_loss.append(loss.cpu().detach().numpy())

  if i % 100 == 0:
    print(i, loss)
    te_MA.append(loss)
    if (len(te_MA) > 1) and ((te_MA[len(te_MA)-1] > te_MA[len(te_MA)-2])):
      break

  
  
# plt.plot(tr_loss)
# plt.plot(te_loss)
# plt.show()

/home/burbank/miniconda3/envs/fastai/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
  0%|                                                                               | 1/5000 [00:01<2:06:25,  1.52s/it]

0 tensor(0.1828, grad_fn=<HuberLossBackward0>)


  2%|█▌                                                                           | 101/5000 [01:46<1:22:35,  1.01s/it]

100 tensor(0.1603, grad_fn=<HuberLossBackward0>)


  4%|███                                                                          | 201/5000 [03:30<1:23:31,  1.04s/it]

...

1700 tensor(0.0138, grad_fn=<HuberLossBackward0>)


 36%|████████████████████████████                                                  | 1801/5000 [33:02<59:58,  1.13s/it]

1800 tensor(0.0132, grad_fn=<HuberLossBackward0>)


 38%|█████████████████████████████▋                                                | 1900/5000 [34:51<56:53,  1.10s/it]

1900 tensor(0.0133, grad_fn=<HuberLossBackward0>)


In [ ]:
ys = []
predicted = []
for y,g,w in te_dataloader:
    model.eval()
    preds = model((g.type(torch.float32),w.type(torch.float32)))
    
    for i in preds:
        predicted.append(i.detach().numpy())
    for x in y:
        ys.append(x.detach().numpy())
        
mypred = np.array(predicted).ravel()
mytarget = np.array(ys)


rmse = []
for p,t in zip(mypred,mytarget):
    if not np.isnan(t[-1]) or np.isnan(p):
        target_scaled = (Y.scaler.inverse_transform(np.array(t).reshape(1,-1))[0][-1])
        pred_scaled = (Y.scaler.inverse_transform(np.array(p).reshape(-1,1)))
        rmse.append(np.sqrt((target_scaled-pred_scaled)**2))
    else:
        pass
print(np.mean(rmse))

2.8937774


In [ ]:
class CNNforA(nn.Module):
    def __init__(self, in_chan = 1 , num_classes=100):
        super(EfficientNet2, self).__init__()

        # Define the convolutional layers
        self.conv1 = nn.Conv1d(in_chan, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm1d(128)

        # Define the depthwise separable convolutional layers
        self.dwconv1 = nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1, groups=128, bias=False)
        self.bn4 = nn.BatchNorm1d(128)

        self.dwconv2 = nn.Conv1d(128, 128, kernel_size=3, stride=2, padding=1, groups=128, bias=False)
        self.bn5 = nn.BatchNorm1d(128)

        # Define the fully connected layer
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # Pass the input through the convolutional layers
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)

        # Pass the input through the depthwise separable convolutional layers
        x = self.dwconv1(x)
        x = self.bn4(x)
        x = F.relu(x)

        x = self.dwconv2(x)
        x = self.bn5(x)
        x = F.relu(x)

        # Flatten the feature maps before passing them through the fully connected layer
        # x = x.view(-1, 128)
        # x = self.fc(x)

        return x

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler


#| export
class newGemDataset():
    """
    Pytorch Dataset which can be used with dataloaders for simple batching during training loops
    """
    def __init__(self,W,Y,G, def_device='cpu'):
        self.W = W
        self.SNP = G
        self.Y = Y
        self.device = def_device
        
    def __len__(self): return self.Y[0].shape[0]

    def __getitem__(self,idx):
      y = self.Y[0][idx]
      e = self.Y[1][idx]
      h = self.Y[2][idx]
      d = self.Y[3][idx]

      #weather
      w = self.W[1][np.where(self.W[0] == e)[0][0]]

      #snp
      g = snp_data[1][:,np.where(snp_data[0] == h)[0][0]]
      return y,g,w


#| export
class ST():
    """
    A class which will hold the secondary trait data for the entire dataset for pre-training purposes
    
    init
        yield_data -> pandas table
        testYear -> e.g. 2019. this will set all data from a given year as the Test Set
    """
    def __init__(self, yield_data, testYear):

        self.Te = yield_data.iloc[([str(testYear) in x for x in yield_data['Env']]),:].reset_index()
        self.Tr = yield_data.iloc[([str(testYear) not in x for x in yield_data['Env']]),:].reset_index()

        self.secondary_traits = [
                'Stand_Count_plants',
                'Pollen_DAP_days',
                'Silk_DAP_days',
                'Plant_Height_cm',
                'Ear_Height_cm',
                #'Root_Lodging_plants',
                #'Stalk_Lodging_plants',
                'Twt_kg_m3',
                'Yield_Mg_ha',
                #'Date_Harvested'
                ]
        
        self.setup_scaler()
        self.scale_data(self.Tr)
        self.scale_data(self.Te)

        self.make_arrays(self.Tr)
        self.make_arrays(self.Te, False)
    def setup_scaler(self):
        ss = MinMaxScaler()
        ss.fit(np.array(self.Tr[self.secondary_traits]))
        self.scaler = ss

    def scale_data(self,df):
        scaled_secondary = self.scaler.transform(np.array(df[self.secondary_traits]))
        for c,i in enumerate(self.secondary_traits):
            #print(i)
            df[i] = scaled_secondary[:,c]
    
    def plot_yields(self):
        for i in self.secondary_traits:
            plt.hist(self.Tr[i],density=True, label='Train',alpha=.5,bins=50)
            plt.hist(self.Te[i],density=True, label='Test',alpha=.5,bins=50)
            plt.legend()
            plt.title(i)
            plt.show()

    def make_arrays(self,df,train=True):
      df = np.array(df[self.secondary_traits]), np.array(df['Env']) , np.array(df['Hybrid']), np.array(df['Date_Planted'])
      if train:
        self.Tr = df
      else:
        self.Te= df

#| export
class newWT():
    """
    A class which will hold the weather data for the entire dataset for training purposes
    
    init
        weather_data -> pandas table
        testYear -> e.g. 2019. this will set all data from a given year as the Test Set
    """
    def __init__(self, weather_data, testYear):
        
        self.Te = weather_data.iloc[([str(testYear) in x for x in weather_data['Year']]),:].reset_index()
        self.Tr = weather_data.iloc[([str(testYear) not in x for x in weather_data['Year']]),:].reset_index()
            
        self.setup_scaler()
        self.scale_data(self.Tr)
        self.scale_data(self.Te)

        self.make_array(self.Tr)
        self.make_array(self.Te,False)
            
    def setup_scaler(self):
        ss = MinMaxScaler()
        ss.fit(self.Tr.select_dtypes('float'))
        self.scaler = ss
            
    def scale_data(self, df):
        fd = df.select_dtypes('float')
        fs = self.scaler.transform(fd)
        df[fd.columns] = fs

    def make_array(self, df,train = True):
      for c,i in enumerate(set(df['Env'])):
        env_weather = np.array(df[df['Env'] == i].iloc[:,4:-1])
        #print(env_weather.shape)
        if c == 0:
          env_order = list([i])
          weather_array =   np.array(df[df['Env'] == i].iloc[:,4:-1])
          weather_array = np.expand_dims(weather_array,axis=0)
        else:
          weather_array = np.vstack((weather_array, env_weather[None,:,:]))
          env_order.append(i)

        if train:
          self.Tr = (np.array(env_order), np.array(weather_array))
        else:
          self.Te = (np.array(env_order), np.array(weather_array))